In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np

In [2]:
train_dir = '/content/drive/My Drive/dsets/StanfordCars/car_data/car_data/train'
test_dir = '/content/drive/My Drive/dsets/StanfordCars/car_data/car_data/test'

In [3]:
tr_transforms = transforms.Compose([transforms.Resize(312),
                                    transforms.CenterCrop(299),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.RandomRotation(15),
                                    transforms.ToTensor(),
                                    transforms.Normalize(
                                        (0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
te_transforms = transforms.Compose([transforms.Resize(312),
                                    transforms.CenterCrop(299),
                                    transforms.ToTensor(),
                                    transforms.Normalize(
                                        (0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [4]:
tr_dataset = torchvision.datasets.ImageFolder(root=train_dir, transform = tr_transforms)
te_dataset = torchvision.datasets.ImageFolder(root=test_dir, transform = te_transforms)

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.get_device_name(device) if device.index == 0 else 'cpu')

Tesla T4


In [6]:
b_s = 32

In [7]:
tr_loader = torch.utils.data.DataLoader(tr_dataset, batch_size=b_s, shuffle=True, pin_memory=True)
te_loader = torch.utils.data.DataLoader(te_dataset, batch_size=b_s, shuffle=True, pin_memory=True)

In [8]:
model = models.resnet101(pretrained=True)
model.fc = nn.Linear(2048, 196)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


In [9]:
model.to(device);

In [10]:
error = nn.CrossEntropyLoss()

In [11]:
lr = 0.01

In [12]:
optimiz = optim.Adam(model.parameters(), lr = lr, betas = (0.9, 0.999))
lr_sr = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimiz, mode='max', patience = 3)

In [13]:
def train(epoch, model, optimizer, lr_scheduler, loss_func):

    def get_lr(optimizer):
            for param_group in optimizer.param_groups:
                return param_group['lr']

    num_ep = 0
    print('Training for {} epoch has started'.format(epoch))
    for i in range(epoch):
        num_ep += 1
        loss_stat = []
        model.train()
        for i, (images, labels) in enumerate(tr_loader):

            train = Variable(images.view(len(labels), 3, 299, 299))
            labels = Variable(labels)

            train, labels = train.to(device), labels.to(device)

            optimizer.zero_grad()

            out = model(train)

            loss = loss_func(out, labels)

            loss.backward()

            optimizer.step()

            loss_stat.append(loss.item())

        model.eval()

        test_acc = []

        with torch.no_grad():

            for i, (images, labels) in enumerate(te_loader):

                images = Variable(images.view(len(labels), 3, 299, 299))

                images, labels = images.to(device), labels.to(device)

                pred = model(images)

                _, pred = torch.max(pred.data, 1)

                acc = 100.0 * (((pred == labels).sum().item()) / labels.size(0))

                test_acc.append(acc)

        model.train()
      
        lr_scheduler.step(np.mean(test_acc))
        
        print('Epoch: {}, Train loss: {}, Val acc: {}%, LR: {}'.format(num_ep, np.mean(loss_stat), np.mean(test_acc), get_lr(optimizer)))

        torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/CarsClassifier/model_params.pt')


    return model

In [ ]:
model = train(140, model, optimiz, lr_sr, error)

Training for 140 epoch has started
